In [1]:
pip install pyttsx3

In [2]:
pip install pywin32

In [3]:
pip install numpy

In [4]:
pip install opencv-python

# Exercise 1.1

<p style="font-size:15px;">In this exercise, (I am required to) create an application that detects and tracks moving objects, specifically cars in Laramie (Wyoming, US). The way the application works is through frame differencing and background subtraction.</p>

<p style="font-size:15px;">Firstly, we need to convert each frame of the recording into grayscale. Using the newly gray fram, we compare it to the first frame of the recording to find the difference. The difference will then be examined - if there are pixels above a certain number, we will map that pixel up to 255, which is the highest value. Finally, the frame is blurred.</p>

<p style="font-size:15px;">Using the final blurred frame, the contours of each moving object is obtained. For each contour, is the area of it is more than a certain value, a box will be drawn around it to identify it as a moving object.</p>

<p style="font-size:14px; color:red;">The function will contain comments to explain the code more in-depth.</p>

In [5]:
import cv2
import numpy as np

In [6]:
# three false positives
def object_detection(file):
    # create VideoCapture object and read frames from input file
    video = cv2.VideoCapture(file)
    # check if video opened successfully
    if (video.isOpened()== False):
        print("Error opening video file.")
    else:
        print("Video opened successfully!")
    # global baseline
    initial_frame = None
    # loop until video completes
    while True:
        # capture frame-by-frame
        check, frame = video.read()
        # if there are no more frames to read, exit
        if frame is None:
            break
        # option to exit with 'q'
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
            
        # [600, 1040] is the height and width of the video
        # we are only interested in the main road which is bound to [265:, 0:]
        main_road = frame[265:600, 0:1040]
        # draw a line to separate the video and main road
        cv2.line(frame, (0, 260), (1040, 260), (0, 0, 255), 5, 8, 0)
        
        # gray conversion
        gray_frame = cv2.cvtColor(main_road, cv2.COLOR_BGR2GRAY)
        # set first captured frame as the baseline
        if initial_frame is None:
            initial_frame = gray_frame
        # find the difference of the current frame from the baseline 
        delta_frame = cv2.absdiff(initial_frame, gray_frame)
        # if a pixel value is more than 40, assign it to 255
        _, threshold_frame = cv2.threshold(delta_frame, 30, 255, cv2.THRESH_BINARY)
        # noise reduction
        blur_frame = cv2.GaussianBlur(threshold_frame, (5, 5), 0)
        
        # identify all the contours in the video
        (contours, _) = cv2.findContours(blur_frame, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        for c in contours:
            # if the contour area is more than a certain value, draw a rectangle around it
            if cv2.contourArea(c) > 3000:
                x, y, w, h, = cv2.boundingRect(c)
                cv2.rectangle(main_road, (x, y), (x + w, y + h), (0, 255, 0), 1)
                
        # show video
        cv2.imshow("CCTV", frame)
        # cv2.imshow("Main Road", main_road)
        # cv2.imshow("Gray Frame", gray_frame)
        # cv2.imshow("Blur Frame", blur_frame)
        # cv2.imshow('Delta frame', delta_frame)   
        # cv2.imshow('Threshold frame', threshold_frame)
        
    # release video object
    video.release()
    # destroy all windows
    cv2.destroyAllWindows()

In [7]:
object_detection('Exercise1_Files/Traffic_Laramie_1.mp4')

Video opened successfully!


## References
<ul>
    <li>
        <a href=https://www.coursera.org/learn/uol-cm3065-intelligent-signal-processing/ungradedLab/0PXRU/7-104-exercise-15-introduction-to-motion-detection-with-opencv-and-python/lab?path=%2Fnotebooks%2FExercises%2FExercise%252015.%2520Introduction%2520to%2520motion%2520detection%2520with%2520OpenCV%2520and%2520Python.ipynb>Coursera Lab</a>
    </li>
</ul>